In [1]:
import pandas as pd
import os

In [2]:
print(os.getcwd())
os.chdir('../raw_data')
print(os.getcwd())

/Users/kumo/hec/courses/data_challenge_uw/Data-Challenge-URW-HEC/notebooks
/Users/kumo/hec/courses/data_challenge_uw/Data-Challenge-URW-HEC/raw_data


In [4]:
data = pd.read_csv('cleaned_data.csv')
data.head(5)

,Unnamed: 0,external_mall_id,block_id,average_dwell_time,median_dwell_time,store_id,store_visits_monthly_average,store_window_flow_monthly_average,store_visits_daily_average,store_window_flow_daily_average,...,store_branch_level_1,store_branch_level_2,store_branch_level_3,store_sales_r12m,store_rent_r12m,store_area,service_charges_r12m,marketing_costs_r12m,real_estate_taxes_r12m,maintenance_works_r12m
0,0,18,11273,249.520051,180.0,9257.0,9810.25,472952.50,509.62,24568.96,...,Jewellery,Jewellery & Watches,Modern Jewellery & Watches,886377.00,NaN,104.0,25034.0,0.0,11957.0,6337.0
1,1,18,11271,190.005359,120.0,9443.0,4891.25,362107.00,254.09,18810.75,...,Fashion apparel,Family Fashion,Family Fashion,20883319.00,NaN,203.0,33465.0,0.0,11957.0,6337.0
2,2,17,4319,1325.216860,780.0,5697.0,1981.23,280719.08,70.37,9970.90,...,Services,Imaterial services,Travel Agency,2339819.57,140492.0,72.0,18000.0,980.0,11957.0,6337.0
3,3,36,1201,621.422549,240.0,7179.0,29753.50,751216.75,1226.95,30978.01,...,Food & Beverage Services,Sweet moments,Confectionery,1129538.00,89064.0,107.0,80035.0,5038.0,11957.0,24846.0
4,4,36,1212,217.833860,120.0,7288.0,6082.50,357884.00,253.44,14911.83,...,Fashion apparel,Men's Fashion,Men's Fashion,340091.00,2181010.0,208.0,69226.0,0.0,11957.0,6337.0


# metrics

## trafic

- marketing efficiency = Window Flow/marketing cost
- Visit Rate: Visitor Count / Window Flow Count

make it based on month

In [5]:
data.columns

Index(['Unnamed: 0', 'external_mall_id', 'block_id', 'average_dwell_time',
       'median_dwell_time', 'store_id', 'store_visits_monthly_average',
       'store_window_flow_monthly_average', 'store_visits_daily_average',
       'store_window_flow_daily_average', 'store_total_visits',
       'store_total_window_flow', 'days_with_traffic_data', 'mall_name',
       'mall_country', 'mall_id', 'total_mall_area', 'store_name',
       'store_year_opened', 'store_year_closed', 'store_branch_level_1',
       'store_branch_level_2', 'store_branch_level_3', 'store_sales_r12m',
       'store_rent_r12m', 'store_area', 'service_charges_r12m',
       'marketing_costs_r12m', 'real_estate_taxes_r12m',
       'maintenance_works_r12m'],
      dtype='object')

In [6]:
identity = ['store_id', 'block_id', 'mall_id', 'external_mall_id']
metrics = data[identity]

In [7]:
# marketing efficiency
metrics['mkt_effc'] = data['store_window_flow_monthly_average']/(data['marketing_costs_r12m']/12) 

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/1346066364.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['mkt_effc'] = data['store_window_flow_monthly_average']/(data['marketing_costs_r12m']/12)


In [8]:
# visit rate
# there are monthly, daily and total 
metrics['monthly_visit/window'] = data['store_visits_monthly_average'] / data['store_window_flow_monthly_average']
metrics['daily_visit/window'] = data['store_visits_daily_average'] / data['store_window_flow_daily_average']
metrics['total_visit/window'] = data['store_total_visits'] / data['store_total_window_flow']

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/1795006489.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['monthly_visit/window'] = data['store_visits_monthly_average'] / data['store_window_flow_monthly_average']
/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/1795006489.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['daily_visit/window'] = data['store_visits_daily_average'] / data['store_window_flow_daily_average']
/var/folders/z9/zf5v62yj5_75v0dw

In [9]:
metrics.head(10)
# since they are almost the same, we only keep one
metrics.drop(['daily_visit/window', 'total_visit/window'], axis=1, inplace=True)
metrics.rename(columns={'monthly_visit/window':'visit_rate'}, inplace=True)

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/1917769999.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics.drop(['daily_visit/window', 'total_visit/window'], axis=1, inplace=True)
/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/1917769999.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics.rename(columns={'monthly_visit/window':'visit_rate'}, inplace=True)


## Sales

In [10]:
# revenue per visit
# time unit, month
metrics['RPV'] = (data['store_sales_r12m']/12)/data['store_visits_monthly_average']

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/2476251328.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['RPV'] = (data['store_sales_r12m']/12)/data['store_visits_monthly_average']


In [ ]:
# revenue per m2
# RPM 
metrics['RPM'] = (data['store_sales_r12m']/12)/data['store_area']

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/156298708.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['RPM'] = (data['store_sales_r12m']/12)/data['store_area']


In [12]:
# Rvenue efficiency
metrics['RE'] = data['store_sales_r12m']/data['store_rent_r12m']

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/3354451396.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['RE'] = data['store_sales_r12m']/data['store_rent_r12m']


## cost

In [15]:
data['total_costs_r12m'] = (data['service_charges_r12m'] +
                              data['marketing_costs_r12m'] +
                              data['real_estate_taxes_r12m'] +
                              data['maintenance_works_r12m'])

In [16]:
# cost per m2
metrics['CPM'] = (data['total_costs_r12m']/12)/data['store_area']
# profit margin 
metrics['PM'] = (data['store_sales_r12m']-data['total_costs_r12m'])/data['store_sales_r12m']

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/4174579882.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['CPM'] = (data['total_costs_r12m']/12)/data['store_area']
/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/4174579882.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['PM'] = (data['store_sales_r12m']-data['total_costs_r12m'])/data['store_sales_r12m']


In [17]:
metrics.head(10)

,store_id,block_id,mall_id,external_mall_id,mkt_effc,visit_rate,RPV,RPM,RE,CPM,PM
0,9257.0,11273,54.0,18,inf,0.020743,7.529344,710.237981,NaN,34.717949,0.951118
1,9443.0,11271,54.0,18,inf,0.013508,355.793833,8572.791051,NaN,21.247537,0.997522
2,5697.0,4319,15.0,17,3437.376490,0.007058,98.416117,2708.124502,16.654468,43.141204,0.984070
3,7179.0,1201,35.0,36,1789.321358,0.039607,3.163600,879.702492,12.682318,94.919003,0.892101
4,7288.0,1212,35.0,36,inf,0.016996,4.659419,136.254407,0.155933,35.064103,0.742657
5,5451.0,425,12.0,15,inf,0.042532,5.162610,378.339812,NaN,7.891048,0.979143
6,4491.0,6726,26.0,22,inf,0.006664,10.158061,891.635659,NaN,90.429264,0.898580
7,1650.0,7313,6.0,30,inf,0.014437,NaN,NaN,NaN,63.532051,NaN
8,11057.0,2224,62.0,24,inf,0.069089,2.313707,326.579904,10.445250,171.837261,0.473828
9,1468.0,4674,4.0,10,inf,0.007416,30.134355,328.098129,NaN,16.252174,0.950466


## engagement

In [18]:
# using median dwell in case outlier effect
metrics['Space_u'] = data['median_dwell_time']/data['store_area']

/var/folders/z9/zf5v62yj5_75v0dwqpq9c7nw0000gn/T/ipykernel_84205/2825203873.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics['Space_u'] = data['median_dwell_time']/data['store_area']


In [19]:
metrics.head(5)

,store_id,block_id,mall_id,external_mall_id,mkt_effc,visit_rate,RPV,RPM,RE,CPM,PM,Space_u
0,9257.0,11273,54.0,18,inf,0.020743,7.529344,710.237981,NaN,34.717949,0.951118,1.730769
1,9443.0,11271,54.0,18,inf,0.013508,355.793833,8572.791051,NaN,21.247537,0.997522,0.591133
2,5697.0,4319,15.0,17,3437.376490,0.007058,98.416117,2708.124502,16.654468,43.141204,0.984070,10.833333
3,7179.0,1201,35.0,36,1789.321358,0.039607,3.163600,879.702492,12.682318,94.919003,0.892101,2.242991
4,7288.0,1212,35.0,36,inf,0.016996,4.659419,136.254407,0.155933,35.064103,0.742657,0.576923


In [21]:
metrics.isna().sum(axis=0)

store_id               0
block_id               0
mall_id                0
external_mall_id       0
mkt_effc               0
visit_rate             0
RPV                  884
RPM                  884
RE                  1661
CPM                    0
PM                   884
Space_u                0
dtype: int64

In [25]:
data.columns

Index(['Unnamed: 0', 'external_mall_id', 'block_id', 'average_dwell_time',
       'median_dwell_time', 'store_id', 'store_visits_monthly_average',
       'store_window_flow_monthly_average', 'store_visits_daily_average',
       'store_window_flow_daily_average', 'store_total_visits',
       'store_total_window_flow', 'days_with_traffic_data', 'mall_name',
       'mall_country', 'mall_id', 'total_mall_area', 'store_name',
       'store_year_opened', 'store_year_closed', 'store_branch_level_1',
       'store_branch_level_2', 'store_branch_level_3', 'store_sales_r12m',
       'store_rent_r12m', 'store_area', 'service_charges_r12m',
       'marketing_costs_r12m', 'real_estate_taxes_r12m',
       'maintenance_works_r12m', 'total_costs_r12m'],
      dtype='object')

In [28]:
data['store_rent_r12m'].isna().sum()

np.int64(985)